# Testing Code

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Setup

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/DISSERTATION

/content/drive/MyDrive/Colab Notebooks/DISSERTATION


In [ ]:
!pip install stable_baselines3 -q

In [ ]:
import gymnasium

# utils imports
import numpy as np
import pandas as pd
import random
from sklearn.preprocessing import MinMaxScaler

# environment imports
import numpy as np
import pandas as pd
import random
import os
import json
import sys
import pickle
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon

# training imports
import torch
from stable_baselines3 import PPO  # project 4
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnNoModelImprovement

# set seed variable
seed = 42

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/DISSERTATION/params.json', 'r') as f:
    params = json.load(f)

## Utils

In [ ]:
# calculate profit
def calculate_profit(position_size, trade_direction, entry_price, exit_price):

    fixed_t_cost = 0.0001
    size_multiplier = 1.0
    position_size = position_size * size_multiplier

    # fixed transaction cost
    t_cost = fixed_t_cost * position_size

    price_change = (exit_price - entry_price) / entry_price

    if trade_direction == 1:
        profit = price_change * position_size
        marginal_return = price_change
    elif trade_direction == -1:
        profit = -price_change * position_size
        marginal_return = -price_change
    else:
        return 0, 0

    trade_profit = profit - t_cost
    marginal_return = price_change - fixed_t_cost

    return trade_profit, marginal_return

# get datasets
def get_data(pair, window, theta):

    formatted_theta = '%.5f' % theta

    train_df = pd.read_parquet(f'/content/drive/MyDrive/Colab Notebooks/DISSERTATION/1_DataTransformation/TransformedData/{formatted_theta}/{pair}/Window_{window}/train.parquet.gzip')
    val_df = pd.read_parquet(f'/content/drive/MyDrive/Colab Notebooks/DISSERTATION/1_DataTransformation/TransformedData/{formatted_theta}/{pair}/Window_{window}/validation.parquet.gzip')
    test_df = pd.read_parquet(f'/content/drive/MyDrive/Colab Notebooks/DISSERTATION/1_DataTransformation/TransformedData/{formatted_theta}/{pair}/Window_{window}/test.parquet.gzip')

    # get ask prices
    train_asks = train_df['Ask'].values
    val_asks = val_df['Ask'].values
    test_asks = test_df['Ask'].values

    # get bid prices
    train_bids = train_df['Bid'].values
    val_bids = val_df['Bid'].values
    test_bids = test_df['Bid'].values

    # train_df, val_df, test_df = normalize_dataframes(train_df, val_df, test_df)
    train_df, val_df, test_df = manual_normalise(train_df, val_df, test_df)

    # temp remove to match old approach
    train_df = train_df.drop(['Direction', 'Ask', 'Bid', 'Spread'], axis=1)
    val_df = val_df.drop(['Direction', 'Ask', 'Bid', 'Spread'], axis=1)
    test_df = test_df.drop(['Direction', 'Ask', 'Bid', 'Spread'], axis=1)

    if 'JPY' in pair:
        train_df[['Start', 'DCC']] = train_df[['Start', 'DCC']] / 100
        val_df[['Start', 'DCC']] = val_df[['Start', 'DCC']] / 100
        test_df[['Start', 'DCC']] = test_df[['Start', 'DCC']] / 100

    # print(train_df.head)
    # print(train_df.columns.to_list())

    train_data = train_df.values
    val_data = val_df.values
    test_data = test_df.values

    data_dict = {
        'train_data': train_data,
        'train_asks': train_asks,
        'train_bids': train_bids,
        'val_data': val_data,
        'val_asks': val_asks,
        'val_bids': val_bids,
        'test_data': test_data,
        'test_asks': test_asks,
        'test_bids': test_bids
    }

    # print("------- Data -------")
    # print(f"Train Shape: {train_data.shape}")
    # print(f"Validation Shape: {val_data.shape}")
    # print(f"Test Shape: {test_data.shape}")

    return data_dict

# percentage change
def pct_change(old_value, new_value):
    change = new_value - old_value
    percentage_change = (change / old_value)
    return percentage_change

# shift by n, new start values are replace by np.nan and end values are discarded
def shift(array, shift):
    return np.concatenate(([np.nan] * shift, array[:-shift]))

# rolling window generator, left over events are discarded
def rolling_window(df, window_size, shift):
    for i in range(0, len(df) - window_size + 1, shift):
        yield df.iloc[i:i+window_size]

# take train, validation and test sets and normalise based on training data
def normalize_dataframes(train_df, val_df, test_df):
    # create a MinMaxScaler object
    scaler = MinMaxScaler()

    # fit the scaler on the train DataFrame
    scaler.fit(train_df)

    # normalize each DataFrame using the fitted scaler
    train_normalized = pd.DataFrame(scaler.transform(train_df), columns=train_df.columns)
    val_normalized = pd.DataFrame(scaler.transform(val_df), columns=val_df.columns)
    test_normalized = pd.DataFrame(scaler.transform(test_df), columns=test_df.columns)

    return train_normalized, val_normalized, test_normalized

def manual_normalise(train_df, val_df, test_df):

    DC_start_end = train_df[['Start', 'DCC']]
    train_transformed = (train_df - train_df.min()) / (train_df.max() - train_df.min())
    train_transformed[['Start', 'DCC']] = DC_start_end

    DC_start_end = val_df[['Start', 'DCC']]
    val_transformed = (val_df - train_df.min()) / (train_df.max() - train_df.min())
    val_transformed[['Start', 'DCC']] = DC_start_end

    DC_start_end = test_df[['Start', 'DCC']]
    test_transformed = (test_df - train_df.min()) / (train_df.max() - train_df.min())
    test_transformed[['Start', 'DCC']] = DC_start_end

    return train_transformed, val_transformed, test_transformed

# trade graph
def plot_trades(theta, pair, window, trades_df, prices):

    x = range(len(prices))
    y = prices

    # create a figure and axes
    fig, ax = plt.subplots(figsize=(40, 10))

    # plot the price curve
    line = ax.plot(x, y)

    if len(trades_df) > 0:

        short_index = trades_df[(trades_df['Trade Type'] == 'Short')]['Trade Index'].values
        long_index = trades_df[trades_df['Trade Type'] == 'Long']['Trade Index'].values

        short_trades = [[i, prices[i]] for i in short_index]
        long_trades = [[i, prices[i]] for i in long_index]

        polygons = []
        buys = long_trades
        sells = short_trades

        # trade market dims
        marker_x = 5
        marker_y = (max(prices) - min(prices)) * 0.02

        for buy in buys:
            buy_x, buy_y = buy
            polygons.append(Polygon(xy=[[buy_x, buy_y], [buy_x-marker_x, buy_y-marker_y], [buy_x+marker_x, buy_y-marker_y]],
                                    closed=True, color='green'))

        for sell in sells:
            sell_x, sell_y = sell
            polygons.append(Polygon(xy=[[sell_x, sell_y], [sell_x+marker_x, sell_y+marker_y], [sell_x-marker_x, sell_y+marker_y]],
                                    closed=True, color='red'))

        for polygon in polygons:
            ax.add_patch(polygon)

    ax.set_xlabel('Time step')
    ax.set_ylabel('Price')
    ax.set_title(f'Window {window}')

    # show the plot
    graph_dir = f'./TradeGraphs/{theta}/{pair}'
    os.makedirs(graph_dir, exist_ok=True)
    plt.savefig(os.path.join(graph_dir, f'Window_{window}.png'))

# metrics class
class Metrics(object):
    def __init__(self, marginal_returns, total_return):
        self.returns = marginal_returns
        self.total_return = total_return

    def risk(self):
        return np.std(self.returns)

    def max_drawdown(self):
        cumulative_returns = [1]
        [cumulative_returns.append(cumulative_returns[-1] * (1 + r)) for r in self.returns]
        try:
            max_drawdown = max([(max(cumulative_returns[:i+1]) - cumulative_returns[i]) /
                            max(cumulative_returns[:i+1]) for i in range(1, len(cumulative_returns))])
        except:
            return 0
        return max_drawdown

    def calmar_ratio(self):
        md = self.max_drawdown()
        if md == 0:
            return self.total_return
        else:
            return self.total_return / md

    def win_rate(self):
        positive_returns = [r for r in self.returns if r > 0]
        try:
            win_rate = len(positive_returns) / len(self.returns)
        except:
            return 0
        return win_rate

    def average_return(self):
        try:
            return np.mean(self.returns)
        except:
            return 0

    def average_pos_returns(self):
        try:
            positive_returns = [r for r in self.returns if r > 0]
            return sum(positive_returns) / len(positive_returns)
        except:
            return 0



# trend class
class Trend(object):
    def __init__(self, direction, DC_start, DCC, OS_end, DC_start_index, DCC_index, OS_end_index):
        self.direction, self.DC_start, self.DCC, self.OS_end = direction, DC_start, DCC, OS_end
        self.DC_start_index, self.DCC_index, self.OS_end_index = DC_start_index, DCC_index, OS_end_index

        self.data_dict = {
                'Direction': self.direction,
                'Start': round(self.DC_start, 6),
                'DCC': round(self.DCC, 6),
                'End': round(self.OS_end, 6),
                'Start Index': round(self.DC_start_index, 6),
                'DCC Index': round(self.DCC_index, 6),
                'End Index': round(self.OS_end_index, 6),
            }

    def __str__(self):
        return str(self.data_dict)

## Environment

In [ ]:
class TradeDirectionalChangeEnv(gymnasium.Env):
    def __init__(self, starting_balance, env_config):

        # get data from config
        data = env_config['data']
        asks = env_config['asks']
        bids = env_config['bids']

        self.full_data = data
        self.full_asks = asks
        self.full_bids = bids

        self.data = self.full_data
        self.asks = self.full_asks
        self.bids = self.full_bids

        # init state params
        self.lag = params['training']['lag']

        # init state
        self.i = self.lag - 1
        self.state = self.data[0:self.i + 1]
        self.ask_price = asks[self.i]
        self.bid_price = bids[self.i]

        # set action and observation space
        self.action_space = gymnasium.spaces.Discrete(2)  # buy, sell
        self.observation_space = gymnasium.spaces.Box(0, 2, shape=self.state.shape)  # DC start, DC end for last 5 timesteps

        # init simulation params
        self.n_prices = len(self.data) - self.lag
        self.balance = starting_balance
        self.entry_price = None
        self.position_size = None
        self.in_position = 0  # -1 for short, 0 for no, 1 for long
        self.trading_log = []
        self.entry_index = 0

    def step(self, action):

        self.i += 1
        self.n_prices -= 1
        self.state = self.data[self.i - (self.lag-1) : self.i + 1]
        self.ask_price = self.asks[self.i]
        self.bid_price = self.bids[self.i]
        self.mid_price = (self.ask_price + self.bid_price) / 2

        # determine if market is in consolidation period
        dccs = self.state[:-1,1]
        ends = self.state[1:,0]
        os_length = ends - dccs

        # reward function
        if self.in_position == 0 and np.sum(os_length) == 0:  # only enter market when in consolidation period
            if action == 0:  #  buy
                self.entry_price = self.mid_price
                self.position_size = self.balance
                self.in_position = 1
                self.entry_index = self.i
                reward = 0

            elif action == 1:  # sell
                self.entry_price = self.mid_price
                self.position_size = self.balance
                self.in_position = -1
                self.entry_index = self.i
                reward = 0

        elif self.in_position == -1:
            if action == 0:  #  buy
                profit, m_return = calculate_profit(self.position_size, self.in_position,
                                          self.entry_price, self.mid_price)  # exit short position at ask price
                self.trading_log.append({'Trade Index': self.entry_index,
                                         'Position Size': self.position_size,
                                         'Trade Type': 'Short',
                                         'Entry Price': self.entry_price,
                                         'Exit Price': self.mid_price,
                                         'Profit': profit,
                                         'Marginal Return': m_return})
                reward = profit
                self.balance += profit
                self.in_position = 0
                self.position_size = None
                self.entry_price = None
            elif action == 1:  # sell
                reward = 0

        elif self.in_position == 1:
            if action == 0:  #  buy or hold
                reward = 0
            elif action == 1:  # sell
                profit, m_return = calculate_profit(self.position_size, self.in_position,
                                          self.entry_price, self.mid_price)  # exit long position as bid price
                self.trading_log.append({'Trade Index': self.entry_index,
                                         'Position Size': self.position_size,
                                         'Trade Type': 'Long',
                                         'Entry Price': self.entry_price,
                                         'Exit Price': self.mid_price,
                                         'Profit': profit,
                                         'Marginal Return': m_return})
                reward = profit
                self.balance += profit
                self.in_position = 0
                self.position_size = None
                self.entry_price = None

        else:
            reward = 0

        # cover final case
        if self.n_prices <= 0 or self.balance <= 0:
            done = True
            if self.in_position != 0:

                if self.in_position == -1:
                    trade_type = 'Short'
                    exit_price = self.mid_price
                elif self.in_position == 1:
                    trade_type = 'Long'
                    exit_price = self.mid_price

                profit, m_return = calculate_profit(self.position_size, self.in_position,
                                            self.entry_price, exit_price)
                reward = profit

                self.trading_log.append({'Trade Index': self.entry_index,
                                         'Position Size': self.position_size,
                                         'Trade Type': trade_type,
                                         'Entry Price': self.entry_price,
                                         'Exit Price': exit_price,
                                         'Profit': profit,
                                         'Marginal Return': m_return})
                self.balance += profit
                self.in_position = 0
        else:
            done = False

        info = {'balance': self.balance,
                'trading_log': self.trading_log}


        return self.state, reward, done, False, info

    def reset(self, starting_balance):

        # get data
        self.data = self.full_data
        self.asks = self.full_asks
        self.bids = self.full_bids

        # reset episode variables
        self.i = self.lag - 1
        self.state = self.data[0:self.i + 1]
        self.ask_price = self.asks[self.i]
        self.bid_price = self.bids[self.i]
        self.n_prices = len(self.data) - self.lag
        self.balance = starting_balance
        self.entry_price = None
        self.position_size = None
        self.in_position = 0  # -1 for short, 0 for no, 1 for long
        self.trading_log = []
        self.entry_index = 0

        return self.state, {}



## Evaluation

In [ ]:
from stable_baselines3 import PPO, A2C, DQN

In [ ]:
class WindowSimulator(object):
    """class to run a single window"""
    def __init__(self, window, starting_balance, experiment_params, mode, model_name):

        self.balance = starting_balance
        self.window = window
        self.experiment_params = experiment_params

        model_obj_dict = {"PPO": PPO, "A2C": A2C, "DQN": DQN}

        # get data
        data_dict = get_data(pair, window, theta)

        if mode == 'Train':
            data = data_dict['train_data']
            asks = data_dict['train_asks']
            bids = data_dict['train_bids']
        elif mode == 'Val':
            data = data_dict['val_data']
            asks = data_dict['val_asks']
            bids = data_dict['val_bids']
        elif mode == 'Test':
            data = data_dict['test_data']
            asks = data_dict['test_asks']
            bids = data_dict['test_bids']

        # generate prices for visualisation later
        self.prices = (asks + bids) / 2

        # load model
        self.model = model_obj_dict[model_name].load(f'/content/drive/MyDrive/Colab Notebooks/DISSERTATION/2_Training/Models/{model_name}/{theta}/{pair}/{model_name}_Window_{window}')
        # self.model = PPO.load(f'../2_Training/Models/{model}/{theta}/{pair}/PPO_Window_{window}')

        # configure environment
        self.trade_env_config = {
                'data': data,
                'asks': asks,
                'bids': bids
                }

    def run_simulation(self):

        # init tracking variables
        total_trades = 0
        trades = []

        # init environment
        self.env = TradeDirectionalChangeEnv(self.balance, self.trade_env_config)
        obs, _ = self.env.reset(self.balance)
        done = False

        # loop over events
        while not done:
            action, _ = self.model.predict(obs, deterministic=True)
            obs, reward, done, _, info = self.env.step(action)
            self.balance += reward

        return self.balance, len(info["trading_log"]), info["trading_log"]


## Run Testing

In [ ]:
# set seeds
seed = 42
random.seed(seed)
np.random.seed(seed)

# set script variables
pair = 'USDCAD'
theta = 0.00023
mode = 'Test'  # 'Val' or 'Test' - always use 'Val' until you wish to get the final resutls that will be published in your dissertation
n_windows = 19  # this is the number of windows you have trained models for
model_name = 'DQN'

# init simulation values
balance = 100

# init logging values
trades = []
pair_marginal_returns = []
result_dict = {'Return (%)': [], 'Risk (%)': [], 'Maximum Drawdown (%)': [], 'Calmar Ratio': [], 'Win Rate (%)': [],
    'Average Return (%)': [], 'Ave. Positive Returns (%)': []}

# loop over each window
for i in range(n_windows):
    window_start_balance = balance

    # create experiment
    trader = WindowSimulator(i, balance, params, mode, model_name)
    balance, n_trades, trade_details = trader.run_simulation()
    window_end_balance = balance

    # log trades
    trades.extend(trade_details)
    marginal_returns = [trade['Marginal Return'] for trade in trade_details]
    pair_marginal_returns.extend(marginal_returns)
    window_return = (window_end_balance - window_start_balance) / window_start_balance
    print(f'Window: {i} Balance: {balance} No.Trades: {n_trades}')

    # calculate window metrics
    metrics = Metrics(marginal_returns, window_return)
    result_dict['Return (%)'].append(window_return * 100)
    result_dict['Risk (%)'].append(metrics.risk() * 100)
    result_dict['Maximum Drawdown (%)'].append(metrics.max_drawdown() * 100)
    result_dict['Calmar Ratio'].append(metrics.calmar_ratio())
    result_dict['Win Rate (%)'].append(metrics.win_rate() * 100)
    result_dict['Average Return (%)'].append(metrics.average_return() * 100)
    result_dict['Ave. Positive Returns (%)'].append(metrics.average_pos_returns() * 100)

    # visualise trades
    window_trades_df = pd.DataFrame(trade_details)
    plot_trades(theta, pair, i, window_trades_df, trader.prices)

# log total metrics
total_return = (balance - 100) / 100  # calcualte from starting balance of 100
full_metrics = Metrics(pair_marginal_returns, total_return)
result_dict['All Trades'] = {}
result_dict['All Trades']['Return (%)'] = total_return * 100
result_dict['All Trades']['Risk (%)'] = full_metrics.risk() * 100
result_dict['All Trades']['Maximum Drawdown (%)'] = full_metrics.max_drawdown() * 100
result_dict['All Trades']['Calmar Ratio'] = full_metrics.calmar_ratio()
result_dict['All Trades']['Win Rate (%)'] = full_metrics.win_rate() * 100
result_dict['All Trades']['Average Return (%)'] = full_metrics.average_return() * 100
result_dict['All Trades']['Ave. Positive Returns (%)'] = full_metrics.average_pos_returns() * 100

print(f'{pair} Total Return: {round(total_return * 100, 2)}% Final Balance: £{round(balance, 2)}')

# log trades
trades_df = pd.DataFrame(trades)
trades_dir = f'./Trades/{model_name}/{theta}/{mode}'
os.makedirs(trades_dir, exist_ok=True)
trades_df.to_csv(os.path.join(trades_dir, f'{pair}.csv'))

# save results
result_dir = f'./Results/{model_name}/{theta}/{mode}'
os.makedirs(result_dir, exist_ok=True)
with open(os.path.join(result_dir, f'{pair}.json'), 'w') as f:
    json.dump(result_dict, f, indent=4)

Output hidden; open in https://colab.research.google.com to view.